# **Problem 5:  Creating an Image Mosaic using OpenCV**

In [1]:
import cv2
from bing_image_downloader import downloader
import os
import numpy as np
import shutil
import random

In [2]:
def download_images(query, limit, temp_dir, final_dir):
    downloader.download(query, limit=limit, output_dir=temp_dir, adult_filter_off=True, force_replace=False, timeout=60)
   
    downloaded_path = os.path.join(temp_dir, query)
    os.makedirs(final_dir, exist_ok=True)
    
    # Retrieve the list of downloaded images
    images = [img for img in os.listdir(downloaded_path) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if not images:
        print("No images found!")
        return
    
    for img in images:
        shutil.move(os.path.join(downloaded_path, img), final_dir)
    
    shutil.rmtree(temp_dir)
    print(f"Images successfully saved in '{final_dir}/'.")

download_images(query="leonardo dicaprio", limit=150, temp_dir="downloads", final_dir="images")

[%] Downloading Images to d:\...AUT\2 DIP\HomeWork\HW1\P_05\downloads\leonardo dicaprio


[!!]Indexing page: 1

[%] Indexed 34 Images on Page 1.


[%] Downloading Image #1 from https://i.pinimg.com/originals/8d/da/31/8dda31e8cf8a85319be7cb2c880a82c1.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://image.tmdb.org/t/p/original/aLUFp0zWpLVyIOgY0scIpuuKZLE.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://image.tmdb.org/t/p/original/wo2hJpn04vbtmh0B9utCFdsQhxM.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://d1jyxxz9imt9yb.cloudfront.net/person/1571/detail_image/regular/LDC-High-Res-Headshot.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://cdn.britannica.com/05/156805-050-4B632781/Leonardo-DiCaprio-2010.jpg
[%] File Downloaded !

[%] Downloading Image #6 from https://prod-images.tcm.com/Master-Profile-Images/LeonardoDiCaprio.jpg
[%] File Downloaded !

[%] Downloading Image #7 from http://images4.fanpop.com/image/photos/19300000

In [3]:
def select_random_target(final_dir="images", target_name="target.jpg"):
    images = [img for img in os.listdir(final_dir) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if not images:
        print("No images found!")
        return
    
    # Select a random image
    selected = random.choice(images)
    shutil.copy(os.path.join(final_dir, selected), target_name)
    print(f"Selected target: {selected}, saved as '{target_name}'.")

select_random_target(final_dir="images", target_name="target.jpg")

Selected target: Image_14.jpg, saved as 'target.jpg'.


In [4]:
def split_target_image(target_path, grid_size=(25, 25)):
    target = cv2.imread(target_path)
    height, width, _ = target.shape
    
    # Calculate the size of each tile
    rows, cols = grid_size
    tile_height, tile_width = height // rows, width // cols  
    
    tiles = []
    positions = [] 

    # Split the image into tiles
    for i in range(rows):
        for j in range(cols):
            y1, y2 = i * tile_height, (i + 1) * tile_height
            x1, x2 = j * tile_width, (j + 1) * tile_width
            tile = target[y1:y2, x1:x2]
            tiles.append(tile)
            positions.append((y1, y2, x1, x2))

    return tiles, positions, (tile_height, tile_width)

tiles, positions, (tile_height, tile_width) = split_target_image("target.jpg")
print(f"Generated {len(tiles)} tiles of size {tile_height}x{tile_width}")

Generated 625 tiles of size 120x80


In [5]:
# Get the average color (BGR)
def average_color(image):
    return np.mean(image, axis=(0, 1)) 

# Find the best matching
def find_best_match(tile, image_folder):
    tile_avg = average_color(tile) 
    best_match, min_distance = None, float("inf")
    tile_size = (tile.shape[1], tile.shape[0])  # (width, height)
    
    for img_name in os.listdir(image_folder):
        img_path = os.path.join(image_folder, img_name)
        img = cv2.imread(img_path)
        
        if img is None:
            continue  

        img_resized = cv2.resize(img, tile_size)  
        img_avg = average_color(img_resized)  
        distance = np.linalg.norm(tile_avg - img_avg)  # Euclidean distance
        
        if distance < min_distance:
            min_distance, best_match= distance, img_resized
    
    return best_match

In [6]:
def create_mosaic(target_path, image_folder, grid_size=(25, 25), output_path="mosaic.jpg"):
    tiles, positions, (tile_height, tile_width) = split_target_image(target_path, grid_size)
    
    target = cv2.imread(target_path)
    if target is None:
        print("Error: Could not load the target image!")
        return
    
    mosaic = np.zeros_like(target)

    # Replace tiles with best matches
    for tile, (y1, y2, x1, x2) in zip(tiles, positions):
        best_match = find_best_match(tile, image_folder)
        
        if best_match is not None:
            mosaic[y1:y2, x1:x2] = best_match
    
    cv2.imwrite(output_path, mosaic)
    print(f"Mosaic saved as {output_path}")

create_mosaic("target.jpg", "images")

Mosaic saved as mosaic.jpg
